In [32]:
'''coding = utf-8'''
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import json

In [2]:
from transformers import pipeline

In [28]:
## the classifier is used to NER.
## 当然这东西花了十多分钟随便搭起来的，简陋了一点
class AbstractClassifier(object):
        
        def __init__(self,model_name,tokenizer_name,token_type):
            self.classifier = pipeline("token-classification",model=model_name,tokenizer=tokenizer_name)
            self.token_type = token_type
        # it will return a index_buf
        # item in index_buf explain
        # index_buf[0]: word start index
        # index_buf[1]: word end index
        # index_buf[2]: word types
        def get_sentence_index(self,sentence):
            ## 感兴趣的可以自行看看result的返回值，或许对你有所帮助
            dic_result = self.classifier(sentence)

            index_buf = []
            count = -1
            has_begin = False
            for item in dic_result:
                if( 'b-' in item['entity'].lower() ):
                    has_begin = True
                    count += 1
                    index_buf.append([item['start'],item['end'],self.token_type])
                if( 'i-' in item['entity'].lower() ):
                    if(has_begin):
                        index_buf[count][1] = item['end']
                if( 'o' == item['entity'].lower() ):
                    has_begin = False
            return index_buf


In [46]:
def merg_or_append(entities, name, entity):
    has_same = False

    for key in entities:
        if(key.lower() == name.lower()):
            has_same = True

            if( key != name ):
                if('alias' not in entities[key]):
                    entities[key]['alias'] = []
                if( name not in entities[key]['alias']):
                    entities[key]['alias'] += [name]
            break
    
    if(not has_same):
        entities[name] = entity

In [47]:
## 用来识别 疾病实体 的分类器
disease_classifier = AbstractClassifier(model_name="alvaroalon2/biobert_diseases_ner",
                                        tokenizer_name="alvaroalon2/biobert_diseases_ner",
                                        token_type="disease")
## 用来识别 药物/化学品实体 的分类器
chemical_classifier = AbstractClassifier(model_name="sschet/biobert_chemical_ner",
                                         tokenizer_name="sschet/biobert_chemical_ner",
                                         token_type="chemical")
## 用来识别 基因/蛋白质实体 的分类器
genetic_classifier = AbstractClassifier(model_name="alvaroalon2/biobert_genetic_ner",
                                        tokenizer_name="alvaroalon2/biobert_genetic_ner",
                                        token_type="genetic")

In [48]:
# 分割为句子（简陋版本）
# 具体的思路就是读取内容，根据特别的符号来分割句子
# 对于一些小数点的数据就会出现问题
input_dir = os.path.abspath('.')
sentences = []
sentence = ""

for root, dirs, files in os.walk(os.path.join(input_dir,'dataset\\txt\\')):
    for file in files:
        line = ""
        fn = os.path.join(root,file)
        isAppend = False
        with open(fn,'r') as file_to_read:
            while True:
                line = file_to_read.readline()
                if not line:
                    ## 文件结尾退出文件
                    break
                else:
                    ## 空格行自动跳过
                    if( line == '\n'):
                        continue
                    start_index = 0
                    for index in range(len(line)):
                        if(line[index] == '.' 
                           or line[index] == ';' 
                           or line[index] == '?'
                           or line[index] == '!'):
                            if( start_index == index ):
                                start_index = index + 1
                                continue
                            if( isAppend ):
                                isAppend = False
                                sentence += line[start_index:index]
                            else:
                                sentence = line[start_index:index]
                            start_index = index + 1
                            sentences.append(sentence)
                            sentence = ""
                        if(line[index] == '\n'):
                            if( not isAppend ):
                                isAppend = True
                            else:
                                sentence += ' '
                            sentence += line[start_index:index]
        
        words = []
        type_strs = []

        ## Get article aentences
        for sentence in sentences:
            ## for each sentence
            index_bufs = []  ## 用来存字符索引的
            index_bufs.append(disease_classifier.get_sentence_index(sentence))
            index_bufs.append(chemical_classifier.get_sentence_index(sentence))
            index_bufs.append(genetic_classifier.get_sentence_index(sentence))

            
            ## 按照以下方式来打印识别到的实体以及类型
            for index_buf in index_bufs:
                for index_pair in index_buf:
                    words.append(sentence[index_pair[0]:index_pair[1]]) 
                    type_strs.append(index_pair[2])
               
        empty_json = {}
        entities = json.loads(json.dumps(empty_json))

        assert(len(words) == len(type_strs))

        for index in range(len(words)):
            entity = {
                "type":type_strs[index]
            }
            merg_or_append(entities,words[index],entity)

        # print(entities)

        with open("./ner_output/entities.json","w") as file:
            json.dump(entities,file)
        break


In [20]:
## 测试句子
sentence = "In 1964, Blumberg and Alter collected blood samples from all over the world for the study of lipoprotein polymorphisms and serendipitously observed an unusual reaction between serumfrom a transfused hemophilic patient and an Australian aborigine, and the new antigen was designated as the “Australian antigen” [2, 3]"
## 用来存字符索引的
index_bufs = []
index_bufs.append(disease_classifier.get_sentence_index(sentence))
index_bufs.append(chemical_classifier.get_sentence_index(sentence))
index_bufs.append(genetic_classifier.get_sentence_index(sentence))

In [ ]:
## 按照以下方式来打印识别到的实体
for index_buf in index_bufs:
    for index_pair in index_buf:
        print("word:\""+ sentence[index_pair[0]:index_pair[1]] + "\" type:" + index_pair[2])

In [45]:
test_json={
    "A1":{
        "type":"disease"
    }
}

test_json["A1"]["alias"] = []
test_json["A1"]["alias"] += ["a1"]
test_json["A1"]["alias"] += ["a2"]

print(test_json)

{'A1': {'type': 'disease', 'alias': ['a1', 'a2']}}
